In [1]:
# Import data

In [1]:
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import numpy as np


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <FB2FD416-6C4D-3621-B677-61F07C02A3C5> /opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/image.so
  Reason: tried: '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/lib/python3.9/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/anaconda3/envs/spatialhackathon/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might b

In [2]:
from sklearn.preprocessing import StandardScaler
# 載入資料
train_image_data = torch.load("../SML_train_dataset.pt")



/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_21962/2522590248.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_image_data = torch.load("../SML_train_d

In [3]:
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler

class importDataset(Dataset):
    def __init__(self, S_tiles, M_tiles, L_tiles, labels, meta_info, normal_coords, slide_edge_indices):
        """
        Custom Dataset to load image tiles, labels, and edge indices for training.
        
        Args:
            S_tiles (list): Small scale image tiles (spot-level features).
            M_tiles (list): Medium scale image tiles (spot-level features).
            L_tiles (list): Large scale image tiles (spot-level features).
            labels (list): Ground truth labels (spot-level cell type compositions).
            meta_info (list): Metadata containing slide_id, x, y coordinates for each spot.
            normal_coords (list): Normalized coordinates for each spot.
            slide_edge_indices (dict): Dictionary containing the slide-level edge indices for each slide.
        """
        self.S_tiles = S_tiles
        self.M_tiles = M_tiles
        self.L_tiles = L_tiles
        self.labels = labels
        self.meta_info = meta_info
        self.normal_coords = normal_coords
        self.slide_edge_indices = slide_edge_indices
        
        # Optional: Apply any additional processing to the data here
        # e.g., scaling, normalization, etc.

    def __len__(self):
        # Return the number of samples (spots) in the dataset
        return len(self.S_tiles)

    def __getitem__(self, idx):
        # Get the spot-level data for the sample at index `idx`
        S_tile = self.S_tiles[idx]
        M_tile = self.M_tiles[idx]
        L_tile = self.L_tiles[idx]
        label = self.labels[idx]
        meta = self.meta_info[idx]  # This is a tuple (slide_id, x, y)
        normal_coord = self.normal_coords[idx]
        
        # Extract slide_id from meta_info (slide_id is at index 0)
        slide_id = meta[0]
        
        # Get the corresponding edge_index for this slide from slide_edge_indices
        edge_index = self.slide_edge_indices[idx]
        
        # Convert the tiles to the correct format (channels-first)
        S_tile = torch.tensor(S_tile, dtype=torch.float).permute(2, 0, 1)  # Convert from (H, W, 3) to (3, H, W)
        M_tile = torch.tensor(M_tile, dtype=torch.float).permute(2, 0, 1)
        L_tile = torch.tensor(L_tile, dtype=torch.float).permute(2, 0, 1)
        
        # Convert label and normal_coord to tensors
        label = torch.tensor(label, dtype=torch.float)
        normal_coord = torch.tensor(normal_coord, dtype=torch.float)
        
        # Return the data in a PyTorch-friendly format as a dictionary
        return {
            'S_tile': S_tile,
            'M_tile': M_tile,
            'L_tile': L_tile,
            'label': label,
            'meta': meta,
            'normal_coord': normal_coord,
            'edge_index': edge_index,  # slide-level edge index
            'slide_id': slide_id  # slide_id for the current sample
        }


In [4]:
# ====== Step 2: 建立 Dataset ======
image_train_dataset = importDataset(
    S_tiles=train_image_data['S_tiles'],
    M_tiles=train_image_data['M_tiles'],
    L_tiles=train_image_data['L_tiles'],
    labels=train_image_data['labels'],
    meta_info=train_image_data['meta_info'],
    normal_coords=train_image_data['normal_coords'],
    slide_edge_indices=train_image_data['edge_indices']
)


In [5]:
def check_dataset_item(dataset, idx=8000):
    item = dataset[idx]

    print("🔍 Checking dataset sample:", idx)

    # Check the center tile (small scale)
    S_tile = item['S_tile']
    print(f"📏 S_tile shape: {S_tile.shape} | dtype: {S_tile.dtype} | min: {S_tile.min():.3f}, max: {S_tile.max():.3f}")
    assert S_tile.ndim == 3 and S_tile.shape[0] == 3, "❌ S_tile shape is incorrect, expected (3, H, W)"

    # Check medium scale tile
    M_tile = item['M_tile']
    print(f"📏 M_tile shape: {M_tile.shape} | dtype: {M_tile.dtype} | min: {M_tile.min():.3f}, max: {M_tile.max():.3f}")
    assert M_tile.ndim == 3 and M_tile.shape[0] == 3, "❌ M_tile shape is incorrect, expected (3, H, W)"

    # Check large scale tile
    L_tile = item['L_tile']
    print(f"📏 L_tile shape: {L_tile.shape} | dtype: {L_tile.dtype} | min: {L_tile.min():.3f}, max: {L_tile.max():.3f}")
    assert L_tile.ndim == 3 and L_tile.shape[0] == 3, "❌ L_tile shape is incorrect, expected (3, H, W)"

    # Check label (cell composition)
    label = item['label']
    print(f"🧬 Label shape: {label.shape} | dtype: {label.dtype}")
    assert label.shape[0] == 35 and label.dtype == torch.float32, "❌ Label should be float32 with length 35"

    # Check meta info (slide_id, x, y coordinates)
        # Check meta info (slide_id, x, y coordinates)
    meta = item['meta']
    normal_coord = item['normal_coord']

    print(f"🧬 slide_id: {meta[0]}, x: {meta[1]}, y: {meta[2]}, nor_coor: {normal_coord}")

    print("✅ All checks passed!")
    
# Example of how to use the function
check_dataset_item(image_train_dataset)


🔍 Checking dataset sample: 8000
📏 S_tile shape: torch.Size([3, 32, 32]) | dtype: torch.float32 | min: 0.694, max: 1.000
📏 M_tile shape: torch.Size([3, 64, 64]) | dtype: torch.float32 | min: 0.463, max: 0.969
📏 L_tile shape: torch.Size([3, 128, 128]) | dtype: torch.float32 | min: 0.220, max: 1.000
🧬 Label shape: torch.Size([35]) | dtype: torch.float32
🧬 slide_id: S_5, x: 1533, y: 1283, nor_coor: tensor([0.7631, 0.8888])
✅ All checks passed!


In [7]:
import torch
from torch.utils.data import Subset
import numpy as np

# 確定數據集的總大小（兩者必須相同）
dataset_size = len(image_train_dataset)  # 同時 image_train_dataset 和 graph_train_dataset 長度應相同

# 設定訓練比例，這裡以 80% 為訓練集，20% 為驗證集
train_ratio = 0.8
split_index = int(np.floor(train_ratio * dataset_size))

# 生成隨機索引
indices = torch.randperm(dataset_size).tolist()

# 將索引拆分為訓練和驗證部分
train_indices = indices[:split_index]
val_indices = indices[split_index:]

# 使用 Subset 根據相同的索引建立子集
image_train_subset = Subset(image_train_dataset, train_indices)
image_val_subset = Subset(image_train_dataset, val_indices)


# 現在，你有兩對 DataLoader，且它們的索引是一一對應的
print(f"✅ Validation set size: {len(image_val_subset)} samples")

✅ Validation set size: 1670 samples


In [9]:
from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, Batch

combined_train_loader = DataLoader(image_train_subset, batch_size=32, shuffle=True)

combined_val_loader = DataLoader(image_val_subset, batch_size=32, shuffle=True)


In [32]:
print(data.x.shape)          # 應該反映圖中節點數
print(data.edge_index.min(), data.edge_index.max())
graph_train_dataset[4]["edge_index"]

torch.Size([32, 299008])
tensor(0) tensor(3985)


tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  5,
          5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,
          7,  7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,
          9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10,
         10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12,
         12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14,
         14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16,
         16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18,
         18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 21,
         21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21],
        [ 7, 18, 12,  2, 10,  8,  3,  6, 21,  8, 16

Poteintial issues:
# 1. my val_set tiles image may be included in the sub_tiles of train_set

Note: Since neighbor tiles are reused across samples, some mild information overlap may exist between train and val sets. However, final test set is completely held out.

# Neural Network

import torch
import torch.nn as nn
from torch_geometric.nn import GraphSAGE, global_mean_pool
from torch_geometric.data import Data

# Define a CNN for each scale (S_tile, M_tile, L_tile)
class SmallScaleCNN(nn.Module):
    def __init__(self):
        super(SmallScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector

class MediumScaleCNN(nn.Module):
    def __init__(self):
        super(MediumScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector

class LargeScaleCNN(nn.Module):
    def __init__(self):
        super(LargeScaleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.conv(x).view(x.size(0), -1)  # Flatten the output for feature vector


class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGEModel, self).__init__()
        # GraphSAGE layer
        self.graphsage = GraphSAGE(input_dim, hidden_dim, aggr='mean')
        self.fc = nn.Linear(hidden_dim, output_dim)  # For predicting 35 cell types

    def forward(self, data):
        # Assuming `data` contains the features and edge_index for the graph
        x, edge_index = data.x, data.edge_index
        x = self.graphsage(x, edge_index)  # Apply GraphSAGE to the features
        x = global_mean_pool(x, data.batch)  # Global pooling for each graph (slide)
        return self.fc(x)  # Predict the cell type composition


class AttentionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AttentionHead, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=input_dim, num_heads=1)
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        # Apply attention mechanism
        attn_output, _ = self.attention(x, x, x)  # Self-attention on the feature vector
        return self.fc(attn_output)


class CellTypePredictionModel(nn.Module):
    def __init__(self, small_cnn, medium_cnn, large_cnn, graphsage_model, attention_head):
        super(CellTypePredictionModel, self).__init__()
        self.small_cnn = small_cnn  # CNN for small-scale patch
        self.medium_cnn = medium_cnn  # CNN for medium-scale patch
        self.large_cnn = large_cnn  # CNN for large-scale patch
        self.graphsage_model = graphsage_model  # GraphSAGE for graph-based feature learning
        self.attention_head = attention_head  # Attention mechanism to focus on important features

    def forward(self, S_tile, M_tile, L_tile, edge_index, normal_coords):
        # Step 1: CNN feature extraction for each scale (S_tile, M_tile, L_tile)
        S_features = self.small_cnn(S_tile)
        M_features = self.medium_cnn(M_tile)
        L_features = self.large_cnn(L_tile)
        
        # Combine features from all scales into one feature vector (F_i)
        combined_features = torch.cat([S_features, M_features, L_features], dim=1)
        
        # Step 2: Create PyG Data object for Graph Neural Network
        data = Data(x=combined_features, edge_index=edge_index, pos=normal_coords)
        
        # Step 3: GraphSAGE layer to learn spatial context-aware features
        graphsage_output = self.graphsage_model(data)
        
        # Step 4: Attention mechanism to focus on the relevant features for cell-type prediction
        attention_output = self.attention_head(graphsage_output.unsqueeze(0))  # Add batch dimension
        return attention_output.squeeze(0)  # Remove batch dimension for final output



# Instantiate the individual CNNs for each scale
small_cnn = SmallScaleCNN()
medium_cnn = MediumScaleCNN()
large_cnn = LargeScaleCNN()

# Instantiate the GraphSAGE model and attention head
graphsage_model = GraphSAGEModel(input_dim=16 + 32 + 64, hidden_dim=256, output_dim=256)
attention_head = AttentionHead(input_dim=256, output_dim=35)  # Predict 35 cell types

# Instantiate the final model
model = CellTypePredictionModel(small_cnn, medium_cnn, large_cnn, graphsage_model, attention_head)

# Example input: Assuming you have a batch of data
S_tile = torch.rand(32, 3, 32, 32)  # Example batch of small-scale image tiles
M_tile = torch.rand(32, 3, 64, 64)  # Example batch of medium-scale image tiles
L_tile = torch.rand(32, 3, 128, 128)  # Example batch of large-scale image tiles
meta_info = [(f"S_{i}", torch.rand(2)) for i in range(32)]  # Dummy metadata for batch
edge_index = torch.randint(0, 32, (2, 50))  # Dummy graph edge indices for each slide

# Forward pass
output = model(S_tile, M_tile, L_tile, meta_info, edge_index)
print("Output shape:", output.shape)  # Should be (batch_size, 35) for the 35 cell types


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphSAGE, global_mean_pool
from torchvision import models

class MultiScaleCNN(nn.Module):
    def __init__(self):
        super(MultiScaleCNN, self).__init__()
        # CNN backbone for extracting features from small, medium, and large patches
        self.conv_s = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_m = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_l = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

    def forward(self, S_tile, M_tile, L_tile):
        # Process each scale of image patch
        f_s = self.conv_s(S_tile).view(S_tile.size(0), -1)
        f_m = self.conv_m(M_tile).view(M_tile.size(0), -1)
        f_l = self.conv_l(L_tile).view(L_tile.size(0), -1)
        # Combine features from all scales
        return torch.cat([f_s, f_m, f_l], dim=1)

class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(GraphSAGEModel, self).__init__()
        # GraphSAGE layer
        self.graphsage = GraphSAGE(input_dim, hidden_dim, num_layers, aggr='mean')
        self.fc = nn.Linear(hidden_dim, output_dim)  # For predicting 35 cell types

    def forward(self, data):
        # Assuming `data` contains the features and edge_index for the graph
        x, edge_index = data.x, data.edge_index
        x = self.graphsage(x, edge_index)  # Apply GraphSAGE to the features
        x = global_mean_pool(x, data.batch)  # Global pooling for each graph (slide)
        return self.fc(x)  # Predict the cell type composition

class MultiLayerAttentionHead(nn.Module):
    def __init__(self, input_dim, output_dim, num_layers=1):
        super(MultiLayerAttentionHead, self).__init__()
        self.attention_layers = nn.ModuleList([
            nn.MultiheadAttention(embed_dim=input_dim, num_heads=1)
            for _ in range(num_layers)
        ])
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        attn_output = x
        for attn_layer in self.attention_layers:
            attn_output, _ = attn_layer(attn_output, attn_output, attn_output)  # Self-attention on the feature vector
        return self.fc(attn_output)


class HEVisium(nn.Module):
    def __init__(self, cnn_backbone, graphsage_model, attention_head):
        super(HEVisium, self).__init__()
        self.cnn_backbone = cnn_backbone  # CNN for multi-scale feature extraction
        self.graphsage_model = graphsage_model  # GraphSAGE for graph-based feature learning
        self.attention_head = attention_head  # Attention mechanism to focus on important features

    def forward(self, S_tile, M_tile, L_tile, edge_index, normal_coords):
        # Step 1: CNN feature extraction
        cnn_features = self.cnn_backbone(S_tile, M_tile, L_tile)
        print("finished cnn")

        # Step 2: Create PyG Data object
        data = Data(x=cnn_features, edge_index=edge_index, pos=normal_coords)
        print("finished PyG")

        # Step 3: GraphSAGE layer to learn spatial context-aware features
        graphsage_output = self.graphsage_model(data)
        print("finished GraphSAGE")

        # Step 4: Attention mechanism to focus on the relevant features for cell-type prediction
        attention_output = self.attention_head(graphsage_output.unsqueeze(0))  # Add batch dimension
        return attention_output.squeeze(0)  # Remove batch dimension for final output

In [13]:
import torch
import numpy as np
from scipy.stats import spearmanr
from tqdm import tqdm

# 🧠 訓練一個 epoch
def train_one_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    pbar = tqdm(dataloader, desc="Training", leave=False)

    for batch in pbar:
        
        S_tile, M_tile, L_tile = batch[0]['S_tile'].to(device), batch[0]['M_tile'].to(device), batch[0]['L_tile'].to(device)
        label = batch[0]['label'].to(device)
        edge_index = batch[1]['edge_index'].to(device)
        normal_coords = batch[0]['normal_coord'].to(device)
        # 優化器清空梯度
        optimizer.zero_grad()
        
        # 前向傳播：計算預測結果
        out = model(S_tile, M_tile, L_tile, edge_index, normal_coords)

        # 計算損失
        loss = loss_fn(out, label)
        loss.backward()  # 反向傳播
        optimizer.step()  # 更新模型參數

        total_loss += loss.item() * S_tile.size(0)
        avg_loss = total_loss / ((pbar.n + 1) * dataloader.batch_size)
        pbar.set_postfix(loss=loss.item(), avg=avg_loss)

    return total_loss / len(dataloader.dataset)


# 📏 驗證模型
def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    total_loss = 0
    preds, targets = [], []

    pbar = tqdm(dataloader, desc="Evaluating", leave=False)

    with torch.no_grad():
        for batch in pbar:
            # 獲取驗證數據
            S_tile, M_tile, L_tile = batch[0]['S_tile'].to(device), batch[0]['M_tile'].to(device), batch[0]['L_tile'].to(device)
            label = batch[0]['label'].to(device)
            edge_index = batch[1]['edge_index'].to(device)
            normal_coords = batch[0]['normal_coord'].to(device)

            # 前向傳播：計算預測結果
            out = model(S_tile, M_tile, L_tile, edge_index, normal_coords)
            loss = loss_fn(out, label)

            total_loss += loss.item() * S_tile.size(0)
            preds.append(out.cpu())
            targets.append(label.cpu())

            pbar.set_postfix(loss=loss.item())

    preds = torch.cat(preds).numpy()
    targets = torch.cat(targets).numpy()

    # 使用 Spearman 相關性來計算預測的準確性
    scores = [spearmanr(preds[:, i], targets[:, i])[0] for i in range(preds.shape[1])]
    spearman_avg = np.nanmean(scores)

    return total_loss / len(dataloader.dataset), spearman_avg



    # ✅ Spearman correlation for each gene
    scores = [spearmanr(preds[:, i], targets[:, i])[0] for i in range(preds.shape[1])]
    spearman_avg = np.nanmean(scores)

    return total_loss / len(dataloader.dataset), spearman_avg

# 🔮 預測
def predict(model, dataloader, device):
    model.eval()
    all_preds = []
    all_meta = []

    with torch.no_grad():
        for batch in dataloader:
            # 獲取測試數據
            S_tile, M_tile, L_tile = batch[0]['S_tile'].to(device), batch[0]['M_tile'].to(device), batch[0]['L_tile'].to(device)
            edge_index = batch[1]['edge_index'].to(device)
            normal_coords = batch[0]['normal_coord'].to(device)

            # 預測
            out = model(S_tile, M_tile, L_tile, edge_index, normal_coords)
            all_preds.append(out.cpu())
            all_meta.extend(batch['meta'])  # 用來存儲測試集的meta信息

    return torch.cat(all_preds).numpy(), all_meta



# callback

In [14]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.verbose = verbose

    def __call__(self, val_loss):
        if self.best_score is None or val_loss < self.best_score:
            self.best_score = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True


import matplotlib.pyplot as plt
from IPython.display import clear_output


def plot_losses(train_losses, val_losses):
    clear_output(wait=True)
    plt.figure(figsize=(8, 4))
    plt.plot(train_losses, label="Train Loss", marker='o')
    plt.plot(val_losses, label="Val Loss", marker='o')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.title("Training vs Validation Loss")
    plt.show()
# 收集資料

In [14]:
import torch
import torch.nn as nn
from torch_geometric.nn import GraphSAGE, global_mean_pool
from torch_geometric.data import Data

class GraphSAGEModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=1):
        super(GraphSAGEModel, self).__init__()
        # 初始化 GraphSAGE 層
        self.graphsage = GraphSAGE(input_dim, hidden_dim, num_layers, aggr='mean')
        self.fc = nn.Linear(hidden_dim, output_dim)  # 用於預測 35 個細胞類型

    def forward(self, data):
        # data 為 PyG 的 Data 物件，包含 x 和 edge_index
        x, edge_index = data.x, data.edge_index
        x = self.graphsage(x, edge_index)  # GraphSAGE 計算節點表示
        # 使用全局池化將每個圖中的節點表示聚合成一個圖表示
        x = global_mean_pool(x, data.batch) if hasattr(data, 'batch') else x.mean(dim=0, keepdim=True)
        return self.fc(x)  # 經過全連接層輸出最終預測

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 創建 MultiScaleCNN 模型實例
model = MultiScaleCNN().to(device)

# 構造假數據
# 假設每個 tile 的尺寸是 64 x 64，並且為 RGB（3 通道）
# 我們可以構造一個 batch，假設 batch_size = 8

edge_index = batch[1]['edge_index'].to(device)
normal_coords = batch[0]['normal_coord'].to(device)
data = Data(x=output, edge_index=edge_index, pos=normal_coords)
input_dim = 128  # 每個節點的特徵維度
hidden_dim = 64
output_dim = 35  # 最終預測 35 個細胞類型
model = GraphSAGEModel(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=1).to(device)

model.eval()  # 設定 evaluation 模式
with torch.no_grad():
    output_GraphSAGEModel = model(data)

print("GraphSAGE Model Output Shape:", output_GraphSAGEModel.shape)


IndexError: Found indices in 'edge_index' that are larger than 31 (got 3985). Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 32) in your node feature matrix and try again.

In [23]:
batch[1]['x'].shape

torch.Size([4096, 128, 3])

In [15]:
import torch
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

# 🔧 設定裝置
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

# 🔧 初始化模型 & 優化器
# 🔧 初始化模型 & 優化器
cnn_backbone = MultiScaleCNN()  # 這是你定義的多尺度CNN
graphsage_model = GraphSAGEModel(input_dim=1024, hidden_dim=512, output_dim=35, num_layers=1)  # 兩層 GraphSAGE
attention_head = MultiLayerAttentionHead(input_dim=256, output_dim=35)  # 假設的 AttentionHead

model = HEVisium(cnn_backbone, graphsage_model, attention_head).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.MSELoss()  # 或者使用其他損失函數，例如 spearman_loss
#loss_fn = spearman_loss

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
early_stopper = EarlyStopping(patience=10)

# 🔧 儲存 log 的設定
log_file = open("training_log.csv", mode="w", newline="")
csv_writer = csv.writer(log_file)
csv_writer.writerow(["Epoch", "Train Loss", "Val Loss", "Val Spearman", "Learning Rate"])

# 🔧 用來畫圖
train_losses = []
val_losses = []


# 🔁 開始訓練
num_epochs = 150
best_val_loss = float('inf')

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, combined_train_loader, optimizer, loss_fn, device)
    val_loss, val_spearman = evaluate(model, combined_val_loader, loss_fn, device)

    # ✅ 儲存最好的模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pt")
        print("✅ Saved best model!")

    # ✅ 調整學習率
    scheduler.step(val_loss)

    # ✅ 寫入 CSV log
    lr = optimizer.param_groups[0]['lr']
    csv_writer.writerow([epoch+1, train_loss, val_loss, val_spearman, lr])

    # ✅ 印 epoch 結果
    print(f"Epoch {epoch+1}/{num_epochs} | loss: {train_loss:.4f} | val_loss: {val_loss:.4f} | ρ: {val_spearman:.4f} | lr: {lr:.2e}")

    # ✅ 更新 loss list 並畫圖
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    plot_losses(train_losses, val_losses)

    # ✅ Early stopping
    early_stopper(val_loss)
    if early_stopper.early_stop:
        print("⛔ Early stopping triggered.")
        break

# ✅ 關閉 log 檔案
log_file.close()


/opt/anaconda3/envs/spatialhackathon/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✅ Using device: mps


RuntimeError: stack expects each tensor to be equal size, but got [2, 118] at entry 0 and [2, 270] at entry 1

In [113]:
# 測試 train_loader 是否返回正確格式的數據
for batch in train_loader:
    print(f"Batch type: {type(batch)}")
    if isinstance(batch, Data):
        print("✅ This batch is a Data object!")
        print(f"Batch keys: {batch.keys()}")
        
        # 打印 S_tile 的 shape 來檢查
        if 'S_tile' in batch:
            print(f"S_tile shape: {batch['S_tile'].shape}")
        else:
            print("⚠️ S_tile not found in batch!")
        
    else:
        print("⚠️ This batch is not a Data object")
    break  # 只檢查第一個批次


Batch type: <class 'abc.DataBatch'>
✅ This batch is a Data object!
Batch keys: ['L_tile', 'label', 'S_tile', 'ptr', 'M_tile', 'edge_index', 'meta', 'normal_coord', 'batch']
S_tile shape: torch.Size([96, 32, 32])


＃# Prediction

In [59]:
# ===== 需要的 Libraries =====
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
import csv

# ===== 載入訓練好的模型權重 =====
from hevisum_model import HEVisumModel
from hevisum_dataset import importDataset

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = VisionMLPModelWithCoord().to(device)
model.load_state_dict(torch.load("best_model.pt"))
model.eval()


✅ Using device: mps


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/3135847424.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model

VisionMLPModelWithCoord(
  (encoder_spot): CNNEncoder(
    (cnn): Sequential(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (10): ReLU()
      (11): AdaptiveAvgPool2d(output_size=(1, 1))
    )
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (linear): Linear(in_features=64, out_features=64, bias=True)
  )
  (encoder_subtiles

In [61]:
# 正確方式
test_data = torch.load("test_dataset.pt")

raw_coords = []
for _meta in data['meta_info']:
    if _meta is not None:
        _, x, y = _meta
    else:
        x, y = 0, 0
    raw_coords.append([x, y])

raw_coords = np.array(raw_coords)


coord_scaler = StandardScaler()
normalized_coords = coord_scaler.fit_transform(raw_coords)

# ====== Step 2: 建立 Dataset ======
test_dataset = importDataset(
    center_tile=test_data['tiles'],
    subtiles=test_data['subtiles'],
    neighbor_tiles=test_data['neighbor_tiles'],
    label=np.zeros((len(test_data['tiles']), 35)),  # dummy label
    meta=normalized_coords
)


test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_54542/1343659088.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  test_data = torch.load("test_dataset.pt")


In [62]:
test_preds, test_meta = predict(model, test_loader, device)


coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch.Size([32, 2])
coords.shape = torch

In [ ]:
test_preds

array([[0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.25503415, 0.13337179, 0.1524582 , ..., 0.04139816, 0.01808124,
        0.03039141],
       ...,
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.29158217, 0.12338457, 0.17179191, ..., 0.03849616, 0.01655579,
        0.02754541],
       [0.23385419, 0.13244098, 0.13941698, ..., 0.04153137, 0.01910294,
        0.03005139]], dtype=float32)

In [66]:
import h5py

# ==== 讀取 test spot index 用於對應 ID ====
with h5py.File("./elucidata_ai_challenge_data.h5", "r") as f:
    test_spots = f["spots/Test"]
    test_spot_table = pd.DataFrame(np.array(test_spots['S_7']))  # Example: S_7


ensemble_df = pd.DataFrame(test_preds, columns=[f"C{i+1}" for i in range(test_preds.shape[1])])
ensemble_df.insert(0, 'ID', test_spot_table.index)
ensemble_df.to_csv("submission.csv", index=False)
print("✅ Saved submission.csv")


✅ Saved submission.csv
